<a href="https://colab.research.google.com/github/Arihant987/LLM/blob/main/Finetune_LLama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
print("Finetuning LLama2 model")

Finetuning LLama2 model


In [15]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

### LLAMA2 Prompt Template

In [4]:
'''
<s>[INST] <<SYS>>
System prompt
<</SYS>>

User prompt [/INST] Model answer </s>
'''

'\n<s>[INST] <<SYS>>\nSystem prompt\n<</SYS>>\n\nUser prompt [/INST] Model answer </s>\n'

In [12]:
model_name="NousResearch/Llama-2-7b-chat-hf"
# already compatible to llama2 template
dataset_name="mlabonne/guanaco-llama2-1k"
new_model="llama-2-7b-finetune"


lora_r=64
lora_alpha=16
lora_dropout=0.1


use_4bit=True

bnb_4bit_compute_dtype="float16"
bnb_4bit_quant_type="nf4"

use_nested_quant=False

outuput_dir="./results"


num_train_epochs=1

fp16=False
bf16=False

per_device_train_batch_size=4
per_device_eval_batch_size=4

gradient_accumulation_steps=1

gradient_checkpointing=True


max_grad_norm=0.3
learning_rate=2e-4
weight_decay=0.001
optim="paged_adamw_32bit"
lr_scheduler_type="cosine"
max_steps=-1
warmup_ratio=0.03


group_by_length=True
save_steps=0
logging_steps=25

max_seq_length=1024
packing=False
device_map={"":0}

In [8]:
dataset=load_dataset(dataset_name,split="train")
compute_dtype=getattr(torch,bnb_4bit_compute_dtype)

bnb_config=BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

if compute_dtype==torch.float16 and use_4bit:
    major, _=torch.cuda.get_device_capability()
    if major>=8:
        print("Your GPU supports bfloat16: accelerate training with bf16=True")

In [9]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache=False
model.config.pretraining_tp=1


tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'


peft_config=LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [13]:
training_arguments=TrainingArguments(
    output_dir=outuput_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


trainer=SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.408000
50,1.657400
75,1.214100
100,1.443300
125,1.177500
150,1.365900
175,1.173900
200,1.466800
225,1.158500
250,1.541600


TrainOutput(global_step=250, training_loss=1.3606754150390625, metrics={'train_runtime': 1409.595, 'train_samples_per_second': 0.709, 'train_steps_per_second': 0.177, 'total_flos': 8755214190673920.0, 'train_loss': 1.3606754150390625, 'epoch': 1.0})

In [17]:
trainer.model.save_pretrained(new_model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [24]:
# Ignore Warnings
logging.set_verbosity(logging.CRITICAL)


prompt="How to own a plane in United States(USA) ?"
pipe=pipeline(task="text-generation",model=model,tokenizer=tokenizer,max_length=200)
result=pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] How to own a plane in United States(USA) ? [/INST] To own a plane in the United States, you will need to meet certain requirements and follow a specific process. everybody has the right to own a plane, but it is not a simple process.

Here are some general steps you can follow to own a plane in the United States:

1. Determine your budget: Owning a plane can be expensive, so it's important to determine how much you can afford to spend.

2. Research the different types of planes: There are many different types of planes available, including small propeller planes, large commercial jets, and everything in between.

3. Find a reputable dealer: Finding a reputable dealer who can help you find the right plane for your needs is important.

4. Obtain financing: Financing a plane can be difficult


In [ ]:
# empty VRAM

del model
del pipe
del trainer
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Reload model in FP16 and merge it with lora weights
base_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model=PeftModel.from_pretrained(base_model,new_model)
model=model.merge_and_unload()

tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

## Pusing to hugging face hub

In [30]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!huggingface-cli login

model.push_to_hub("",check_pr=True)

tokenizer.push_to_hub("",check_pr=True)